In [29]:
import json
import pke
import numpy as np
import string
from nltk.corpus import stopwords
# load the abstractive summerizer
from transformers import pipeline
summarizer = pipeline("summarization")

In [7]:
# load data
f = open('/home/ubuntu/pubmed/dataset/test.json')
total = json.load(f)
f.close()

In [31]:
# a simple evaluation that counts the number of exact match strings of two lists
def match(list1, list2):
    same = set(list1).intersection(set(list2))
    return len(same)

In [ ]:
# the result array [@5, @10, @15] stores the result of extracting 5, 10 and 15 keyphrases 
multirank_ground_truth = np.zeros(3)
multirank_matched = np.zeros(3)
multirank_extracted = np.zeros(3)

# write to file to save extracted keywords
f = open('/home/ubuntu/pubmed/abstractive/multirank.json', 'w')
final = {}
abstracted = {}
# write to file to save summarized abstract and title
f1 = open("abstract_text.json",'w')


for count, body in total.items():
    abstract = body['abstract']
    title = body['title']
    kwds_in = body['keywords_in']
    kwds_not_in = body['keywords_not_in']
    text = title + abstract
    # use summarizer to summarize
    try:
        summed_text = summarizer(text, max_length = 200, min_length = 20, do_sample = False)[0]["summary_text"]
    except:
        continue
    # use pke to extract kwds from summarized text
    extractor = pke.unsupervised.MultipartiteRank()

    extractor.load_document(input=summed_text)

    pos = {'NOUN', 'PROPN', 'ADJ'}
    stoplist = list(string.punctuation)
    stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
    stoplist += stopwords.words('english')
    extractor.candidate_selection(pos=pos, stoplist=stoplist)

    extractor.candidate_weighting(alpha=1.1,
                                  threshold=0.74,
                                  method='average')
    # extract top 5, 10, 15 kwds
    temp = {5:[], 10:[], 15:[]}
    for nn in [5,10,15]:
        keyphrases = extractor.get_n_best(n=nn)
        keyphrases = [i[0] for i in keyphrases]
        index = (int)(nn/5-1)
        multirank_extracted[index] += nn
        multirank_ground_truth[index] += len(kwds_not_in)
        multirank_matched[index] += match(kwds_not_in, keyphrases)
        temp[nn] = keyphrases
    final[count] = temp
    abstracted[count] = summed_text
json.dump(final, f, indent = 4)
json.dump(abstracted, f1, indent = 4)
f1.close()
f.close()                  
multirank_precision = multirank_matched / multirank_extracted
multirank_recall = multirank_matched / multirank_ground_truth
multirank_f1 = 2 * multirank_precision * multirank_recall / (multirank_precision + multirank_recall)


In [38]:
print(multirank_precision)
print(multirank_recall)
print(multirank_f1)

[0.00174413 0.00121127 0.00085622]
[0.00406585 0.00564732 0.00598799]
[0.0024411  0.0019947  0.00149822]
